En esta parte se utilizará otro conjunto muy conocido de datos para aprendizaje, que consiste también de textos en inglés. Se trata de un conjunto de noticias de prensa de la Agencia Reuters, *Reuters-21578 *, https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

Las noticias han sido categorizadas a mano, con etiquetas de varios tipos, incluyendo *temas, lugares, organizaciones, personas* y otros criterios. Están en formato SGML, las etiquetas aparecen embebidas en el texto. Existe documentación en el archivo README de la distribución.

Nos interesaremos en el tipo de etiqueta *temas (en inglés en los textos topics)*. El objetivo es nuevamente aprender los temas a partir de los textos de las noticias, con la diferencia de que este es un problema multi-etiqueta : cada noticia puede tener varios temas. En vez de resolver el problema multi-etiqueta inicial (con más de 100 tópicos distintos) les pedimos que lo transformen según las siguientes simplificaciones: considere solo los 3 temas más frecuentes, y transforme el problema multi-etiqueta en 3 problemas de clasificación binaria

No se realiza para esta parte una especificación detallada, sino que les pedimos a Uds. que armen los pasos de una solución y definan el detalle del notebook para esta parte. Mínimamente se espera que procesen la entrada SGML, y encuentren algún modo de enfocar el problema multi-etiqueta en la versión simplificada. Se debe proponer algún modo de tratar el texto, con eventuales mejoras respecto a la vectorización de la parte 1. Se deben aplicar clasificadores y medir su performance, mínimamente con precision, recall y medida-F. Finalmente, se espera una discusión detallada de todo lo realizado y eventuales propuestas de mejora.

Puntos a tener en cuenta del conjunto de datos:
* En Topics dentro de la etiqueta <REUTERS> puede haber:
    * YES: Tiene al menos una categoría(puede existir algún error y no tener ningúna categoria)
    * NO: No tiene categoría(puede existir algún error y tener alguna categoria, supuestamente en esta versión no sucede)
    * BYPASS: No está indexado, no se verificó la categoría
* Hay etiquetas que dividen el conjunto en train y test
* Etiquetas:
    * <TOPICS>, </TOPICS> [ONCE, SAMELINE]: Lista de categorias. Si hay alguna estan delimitadas por <D></D>.
    * <UNKNOWN>, </UNKNOWN> metadatos desconocidos
    * <TEXT>, </TEXT> cuerpo del texto

##1

Se cargan los archivos a memoria, con la librería BeautifulSoup se parsea cada documento como xml dividiendo los documentos por noticias.

In [1]:
import glob
#Cargo los archivos como lista de strings en documentos
lista_nombres_archivo = glob.glob('reuters21578/*.sgm')
documentos = []
for nombre_archivo in lista_nombres_archivo:
    with open (nombre_archivo, "r") as archivo:
        documento = archivo.read()
        documentos.append(documento)

In [2]:
from BeautifulSoup import BeautifulSoup

raw_reuter_news = []
for documento in documentos:
    #coloco el contenido de los tags 'reuters' en otra lista: raw_reuter_news
    #esa lista contendra las noticias separadas en formato raw
    soup = BeautifulSoup(documento)
    for raw_reuter in soup('reuters'):
        raw_reuter_news.append(raw_reuter)

In [3]:
#verifico cantidad
print len(raw_reuter_news)
assert(21578 == len(raw_reuter_news))

21578


##2

Se cuentan la cantidad de noticias por tema y se consideran los temas más frecuentes.

In [5]:
raw_news_x_topics = {}
for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    for topics_soup in raw_reuter_soup.topics:
        #para cada topic en la lista de topics 
        #agrego la noticia a un diccionario clave topic y valor lista de noticias con ese topic.
        for topic_soup in topics_soup:
            try:
                #ya existe la lista.
                raw_news_x_topics[topic_soup].append(raw_reuter_soup)
            except KeyError:
                #es el primer valor
                raw_news_x_topics[topic_soup] = [raw_reuter_soup]

In [6]:
#ordeno el diccionario por cantidad de noticias en cada topic.
sorted_topics = sorted(raw_news_x_topics, key=lambda topic: len(raw_news_x_topics[topic]), reverse=True)
#los tres topics mas usados
primeros_topics = sorted_topics[:3]
for topic in primeros_topics:
        print topic,len(raw_news_x_topics[topic])

earn 3987
acq 2448
money-fx 801


##3

Se eliminan los metadatos y se separan los conjuntos de entrenamiento y test, para ésto se considera el atributo "LEWISPLIT". De la noticia se considera en principio sólo el texto dentro de la tag "body", las noticias que no contienen texto válido en el body se descartan.

In [9]:
#ya no necesito los metadatos
#separo en train y test segun el criterio lewissplit
#solo body
X_train = []
X_test = []
X_unused = []

#aca dejo info mas completa por si es necesario
#incluye fecha y titulo
X_train_plus = []
X_test_plus = []
X_unused_plus = []

#para etiqueta EARN
Y1_train = []
Y1_test = []
Y1_unused = []

#para etiquete ACQ
Y2_train = []
Y2_test = []
Y2_unused = []

#para etiqueta MONEY-FX
Y3_train = []
Y3_test = []
Y3_unused = []

#cuento los body vacios
empty_body = 0
empty_body_news = []

for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    topics_de_la_noticia = []
    for topics_soup in raw_reuter_soup.topics:
        for topic_soup in topics_soup:
            #para cada topic en la lista de topics 
            topics_de_la_noticia.append(topic_soup)
    #veo si tiene las primeras categorias
    es_earn = 1 if u'earn' in topics_de_la_noticia else 0
    es_acq = 1 if u'acq' in topics_de_la_noticia else 0
    es_money_fx = 1 if u'money-fx' in topics_de_la_noticia else 0
    #extraigo contenido
    body = raw_reuter_soup.find('body').text if raw_reuter_soup.find('body') is not None else None
    title = raw_reuter_soup.find('title').text if raw_reuter_soup.find('title') is not None else None
    date = raw_reuter_soup.find('dateline').text if raw_reuter_soup.find('dateline') is not None else None
    #a veces no hay body
    if body is not None: 
        #conjunto entrenamiento
        if raw_reuter_soup['lewissplit'] == 'TRAIN':
            X_train.append(body)
            X_train_plus.append('\n'.join([title,body,date]))
            Y1_train.append(es_earn)
            Y2_train.append(es_acq)
            Y3_train.append(es_money_fx)
        #conjunto test
        elif raw_reuter_soup['lewissplit'] == 'TEST':
            X_test.append(body)
            X_test_plus.append('\n'.join([title,body,date]))
            Y1_test.append(es_earn)
            Y2_test.append(es_acq)
            Y3_test.append(es_money_fx)
        #otro
        else:
            X_unused.append(body)
            X_unused_plus.append('\n'.join([title,body,date]))
            Y1_unused.append(es_earn)
            Y2_unused.append(es_acq)
            Y3_unused.append(es_money_fx)
    else: 
        empty_body = empty_body + 1
        empty_body_news.append(raw_reuter_soup)

In [17]:
#verifico cantidad noticias
news_n_topics = X_train + X_test + X_unused
total = len(news_n_topics) + empty_body
print "Total: ", total
assert(21578 == total)
print "Noticias con body: ",len(news_n_topics)
#verifico cantidad earn
print "EARN: ",sum(Y1_train+Y1_test+Y1_unused)
#verifico cantidad acq
print "ACQ: ",sum(Y2_train+Y2_test+Y2_unused)
#verifico cantidad money-fx
print "MONEY-FX", sum(Y3_train+Y3_test+Y3_unused)

len_X_train = len(X_train)
len_X_test = len(X_test)
len_X_unused = len(X_unused)

print "Train: ",len_X_train
print "Test: ",len_X_test
print "Unused: ",len_X_unused

assert(len(news_n_topics)==len_X_train+len_X_test+len_X_unused)

assert(len_X_train==len(Y1_train))
assert(len_X_train==len(Y2_train))
assert(len_X_train==len(Y3_train))

assert(len_X_test==len(Y1_test))
assert(len_X_test==len(Y2_test))
assert(len_X_test==len(Y3_test))

Total:  21578
Noticias con body:  19043
EARN:  3776
ACQ:  2210
MONEY-FX 684
Train:  12865
Test:  5458
Unused:  720


In [21]:
#Organizacion de experimentos
Experimentos = [
    ("EARN",Y1_train, Y1_test),
    ("ACQ",Y2_train, Y2_test),
    ("MONEY-FX",Y3_train, Y3_test)
]

In [22]:
#Vectorizacion
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect2 = CountVectorizer(min_df=1)
tidf_vect = TfidfVectorizer()
hash_vect = HashingVectorizer(non_negative=True)
Vectorizadores = [    
    ("TfidfVectorizer", tidf_vect),
    ("CountVectorizer", count_vect),
    ("CountVectorizer2", count_vect2),
    ("HashingVectorizer", hash_vect)
]

In [23]:
#Comparación de diferentes clasificadores
#Fuente: http://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html

# Creación de clasificadores
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

"""
lr = LogisticRegression()
gnb = GaussianNB()
svc = LinearSVC(C=1.0)
rfc = RandomForestClassifier(n_estimators=100)
"""
multiNB = MultinomialNB()
sgd = SGDClassifier()
perceptron = Perceptron()
passiveaggclass = PassiveAggressiveClassifier()
passiveaggclass2 = PassiveAggressiveClassifier(loss='hinge',C=1.0)
passiveaggclass3 = PassiveAggressiveClassifier(loss='squared_hinge',C=1.0)

Clasificadores = [    
    ("MultinomialNB", multiNB),
    ("SGD", sgd),
    ("Perceptron", perceptron),
    ("SGDClassifier", sgd),
    ("Passive-Aggressive I", passiveaggclass),
    ("Passive-Aggressive II", passiveaggclass2),
    ("Passive-Aggressive III", passiveaggclass3)
]

In [26]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
resultados = []

#Funcion que recibe 
#X conj entrenamiento
#Y conj test
#V conj vectorizadores
#C conj clasificadores
def run(train_data,train_target,test_data,test_target,V,C,matrix = True):
    for nomvect, vectorizador in V:
        #Vectorizo
        X_train = vectorizador.fit_transform(train_data)        
        X_test = vectorizador.transform(test_data)
        for nomclf,clasificador in C:
            print "Metrica F1 con los siguientes parametros"
            print "Clasificador: " + nomclf + ", Vectorizador: " + nomvect   
            if nomclf == 'RandomForest':
                X_train = X_train.toarray()
                X_test = X_test.toarray()
            #Entreno
            clasificador.fit(X_train, train_target)
            #Predigo Test
            prediccion = 0
            prediccion = clasificador.predict(X_test)  
            #Resultados
            print metrics.f1_score(test_target, prediccion, average='weighted')
            if (matrix):
                print confusion_matrix(test_target,prediccion)

In [27]:
for nomcat,y_train,y_test in Experimentos:
    print "Categoria: " + nomcat
    run(X_train,y_train,X_test,y_test,Vectorizadores,Clasificadores)

Categoria: EARN
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB, Vectorizador: TfidfVectorizer
0.862068965517
[[4237  176]
 [ 120  925]]
Metrica F1 con los siguientes parametros
Clasificador: SGD, Vectorizador: TfidfVectorizer
0.84827879303
[[4103  310]
 [  47  998]]
Metrica F1 con los siguientes parametros
Clasificador: Perceptron, Vectorizador: TfidfVectorizer
0.832813196612
[[4149  264]
 [ 111  934]]
Metrica F1 con los siguientes parametros
Clasificador: SGDClassifier, Vectorizador: TfidfVectorizer
0.84827879303
[[4103  310]
 [  47  998]]
Metrica F1 con los siguientes parametros
Clasificador: Passive-Aggressive I, Vectorizador: TfidfVectorizer
0.821052631579
[[4114  299]
 [ 109  936]]
Metrica F1 con los siguientes parametros
Clasificador: Passive-Aggressive II, Vectorizador: TfidfVectorizer
0.821052631579
[[4114  299]
 [ 109  936]]
Metrica F1 con los siguientes parametros
Clasificador: Passive-Aggressive III, Vectorizador: TfidfVectorizer
0.821760283061
[[4126  

/usr/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import confusion_matrix
resultados = []

for nomclf,clasificador in Clasificadores:
    for nomvect, vectorizador in Vectorizadores:
        print "Metrica F1 con los siguientes parametros"
        print "Clasificador: " + nomclf
        """
        pipeline = Pipeline([
              ('vectorizador', vectorizador),
              ('clasificador', clasificador)
            ])
        """
        X_V_train = vectorizador.fit_transform(X_train)
        X_V_test = vectorizador.transform(X_test)
        print "Vectorizador: " + nomvect 
        pred=[]
        i=0
        for nomcat,y_train,y_test in ConjuntoEntrenamiento:
            #entreno Y2
            clasificador.fit(X_V_train, y_train)
            #predigo test
            prediccion = 0
            prediccion = clasificador.predict(X_V_test)  
            print "Categoria: " + nomcat
            print metrics.f1_score(y_test, prediccion, average='weighted')

Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB
Vectorizador: TfidfVectorizer
Categoria: EARN
0.862068965517
Categoria: ACQ
0.00928792569659
Categoria: MONEY-FX
0.0
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB
Vectorizador: CountVectorizer
Categoria: EARN
0.81619165634
Categoria: ACQ
0.627021387585
Categoria: MONEY-FX
0.688
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB
Vectorizador: CountVectorizer2
Categoria: EARN
0.81619165634
Categoria: ACQ
0.627021387585
Categoria: MONEY-FX
0.688
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB
Vectorizador: HashingVectorizer
Categoria: EARN
0.801407742584
Categoria: ACQ
0.0
Categoria: MONEY-FX
0.0
Metrica F1 con los siguientes parametros
Clasificador: SGD
Vectorizador: TfidfVectorizer
Categoria: EARN
0.846282372598
Categoria: ACQ
0.81220657277
Categoria: MONEY-FX
0.610878661088
Metrica F1 con los siguientes parametros
Clasificador: SGD
Vectorizador: CountVector

/usr/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1249: UserWarning: The sum of true positives and false positives are equal to zero for some labels. Precision is ill defined for those labels [1]. The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [1]. 
  average=average)


In [15]:
#print Y1_train
#print X_V_train

In [16]:
"""from sklearn import metrics
from sklearn.metrics import confusion_matrix

clf = MultinomialNB()
clf2 = SGDClassifier()
clf3 = MultinomialNB()
#entreno Y1
clf.fit(X_V_train, Y1_train)
#predigo test
pred = clf.predict(X_V_test)
print "EARN: ",metrics.f1_score(Y1_test, pred, average='weighted')

#entreno Y2
clf2.fit(X_V_train, Y2_train)
#predigo test
pred2 = clf2.predict(X_V_test)
print "ACQ: ",metrics.f1_score(Y2_test, pred2, average='weighted')

#entreno Y3
clf3.fit(X_V_train, Y3_train)
#predigo test
pred3 = clf3.predict(X_V_test)
print "MONEY-FX: ",metrics.f1_score(Y3_test, pred3, average='weighted')"""

'from sklearn import metrics\nfrom sklearn.metrics import confusion_matrix\n\nclf = MultinomialNB()\nclf2 = SGDClassifier()\nclf3 = MultinomialNB()\n#entreno Y1\nclf.fit(X_V_train, Y1_train)\n#predigo test\npred = clf.predict(X_V_test)\nprint "EARN: ",metrics.f1_score(Y1_test, pred, average=\'weighted\')\n\n#entreno Y2\nclf2.fit(X_V_train, Y2_train)\n#predigo test\npred2 = clf2.predict(X_V_test)\nprint "ACQ: ",metrics.f1_score(Y2_test, pred2, average=\'weighted\')\n\n#entreno Y3\nclf3.fit(X_V_train, Y3_train)\n#predigo test\npred3 = clf3.predict(X_V_test)\nprint "MONEY-FX: ",metrics.f1_score(Y3_test, pred3, average=\'weighted\')'